In [13]:
# Configs
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False
    print("Not running as a Colab notebook")

Not running as a Colab notebook


In [ ]:
# Imports

# Standard imports
import torch
import numpy as np
import matplotlib.pyplot as plt

# Ollama
# import ollama

# HuggingFace
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

from huggingface_hub import hf_hub_download # ???


In [15]:
# Ollama usage

# Basic usage
# response = ollama.chat(model='deepseek-r1:8b', messages=[
#     {
#         'role': 'user',
#         'content': 'Solve this step by step: What is 25 x 16?'
#     }
# ])
# print(response['message']['content'])

In [ ]:
def load_r1_model(): pass

def load_llama_scope_sae(): pass

In [ ]:
gsm8k_dataset = load_dataset("gsm8k", "main")
print(gsm8k_dataset)

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})


In [12]:
print("Question")
print(gsm8k_dataset["train"][0]["question"])
print("Answer")
print(gsm8k_dataset["train"][0]["answer"])

Question
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer
Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
